# LIB

In [1]:
!pip install catboost 
!pip install category_encoders
!pip install optuna

In [2]:
import pandas as pd
import numpy as np
import warnings, random
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PowerTransformer,LabelEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.cluster import KMeans
from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import StratifiedKFold,train_test_split
import optuna
from optuna.samplers import TPESampler
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

from sklearn.ensemble import RandomForestClassifier

# Load Data

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
# path = '/content/drive/MyDrive/cp1/'
# train = pd.read_csv(path + 'train2.csv')
# test = pd.read_csv(path + 'test2.csv')

In [4]:
train = pd.read_csv('./train2.csv')
test = pd.read_csv('./test2.csv')

In [5]:
train.head()

,Unnamed: 0,id,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,...,email,occyp_type,family_size,begin_month,credit,YEARS_BIRTH,Age_group,YEARS_EMPLOYED,EMPLOYED_group,income_avg
0,0,F138994709202500,F,N,N,0,3948.75,Commercial associate,Higher education,Married,...,0,NAN,2.0,6.0,1.0,39,30,12,10~15,1974.375
1,1,F113801540247500,F,N,Y,1,4826.25,Commercial associate,Secondary / secondary special,Civil marriage,...,1,Laborers,3.0,5.0,1.0,32,30,4,0~5,1608.750
2,2,M190874434450000,M,Y,Y,0,8775.00,Working,Higher education,Married,...,0,Managers,2.0,22.0,2.0,53,50,12,10~15,4387.500
3,3,F150882092202500,F,N,Y,0,3948.75,Commercial associate,Secondary / secondary special,Married,...,0,Sales staff,2.0,37.0,0.0,42,40,5,5~10,1974.375
4,4,F150372105157500,F,Y,Y,0,3071.25,State servant,Higher education,Married,...,0,Managers,2.0,26.0,2.0,42,40,5,5~10,1535.625


In [6]:
train.drop(['Unnamed: 0'],axis=1,inplace=True)
train.head()

,id,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,...,email,occyp_type,family_size,begin_month,credit,YEARS_BIRTH,Age_group,YEARS_EMPLOYED,EMPLOYED_group,income_avg
0,F138994709202500,F,N,N,0,3948.75,Commercial associate,Higher education,Married,Municipal apartment,...,0,NAN,2.0,6.0,1.0,39,30,12,10~15,1974.375
1,F113801540247500,F,N,Y,1,4826.25,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,...,1,Laborers,3.0,5.0,1.0,32,30,4,0~5,1608.750
2,M190874434450000,M,Y,Y,0,8775.00,Working,Higher education,Married,House / apartment,...,0,Managers,2.0,22.0,2.0,53,50,12,10~15,4387.500
3,F150882092202500,F,N,Y,0,3948.75,Commercial associate,Secondary / secondary special,Married,House / apartment,...,0,Sales staff,2.0,37.0,0.0,42,40,5,5~10,1974.375
4,F150372105157500,F,Y,Y,0,3071.25,State servant,Higher education,Married,House / apartment,...,0,Managers,2.0,26.0,2.0,42,40,5,5~10,1535.625


In [7]:
test.drop(['Unnamed: 0'],axis=1,inplace=True)
test.head()

,id,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,...,phone,email,occyp_type,family_size,begin_month,YEARS_BIRTH,Age_group,YEARS_EMPLOYED,EMPLOYED_group,income_avg
0,M219900112500,M,Y,N,0,2193.750,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,...,1,0,No work experience,2.0,60.0,61,60,No work experience,0~5,1096.875
1,F189648671135000,F,N,Y,0,2632.500,State servant,Higher education,Married,House / apartment,...,1,0,Core staff,2.0,36.0,52,50,23,20~25,1316.250
2,F1588721769372,F,N,Y,0,1352.754,Working,Secondary / secondary special,Married,House / apartment,...,1,0,Laborers,2.0,40.0,44,40,0,0~5,676.377
3,M192702531112500,M,Y,N,0,2193.750,Commercial associate,Secondary / secondary special,Married,House / apartment,...,0,0,Drivers,2.0,41.0,53,50,6,5~10,1096.875
4,F178229385225000,F,Y,Y,0,4387.500,State servant,Higher education,Married,House / apartment,...,0,0,Managers,2.0,8.0,49,40,25,25~30,2193.750


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26440 entries, 0 to 26439
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              26440 non-null  object 
 1   gender          26440 non-null  object 
 2   car             26440 non-null  object 
 3   reality         26440 non-null  object 
 4   child_num       26440 non-null  int64  
 5   income_total    26440 non-null  float64
 6   income_type     26440 non-null  object 
 7   edu_type        26440 non-null  object 
 8   family_type     26440 non-null  object 
 9   house_type      26440 non-null  object 
 10  work_phone      26440 non-null  int64  
 11  phone           26440 non-null  int64  
 12  email           26440 non-null  int64  
 13  occyp_type      26440 non-null  object 
 14  family_size     26440 non-null  float64
 15  begin_month     26440 non-null  float64
 16  credit          26440 non-null  float64
 17  YEARS_BIRTH     26440 non-null 

# category / numerical columns

In [9]:
drop_c=['work_phone','phone','email','id','YEARS_EMPLOYED']
train.drop(drop_c,axis=1,inplace=True)
test.drop(drop_c,axis=1,inplace=True)

In [10]:
numerical_feats = train.dtypes[train.dtypes != "object"].index.tolist()
numerical_feats.remove('credit')
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = train.dtypes[train.dtypes == "object"].index.tolist()
print("Number of Categorical features: ", len(categorical_feats))

Number of Numerical features:  7
Number of Categorical features:  9


In [11]:
train.columns

Index(['gender', 'car', 'reality', 'child_num', 'income_total', 'income_type',
       'edu_type', 'family_type', 'house_type', 'occyp_type', 'family_size',
       'begin_month', 'credit', 'YEARS_BIRTH', 'Age_group', 'EMPLOYED_group',
       'income_avg'],
      dtype='object')

In [12]:
numerical_feats

['child_num',
 'income_total',
 'family_size',
 'begin_month',
 'YEARS_BIRTH',
 'Age_group',
 'income_avg']

In [13]:
categorical_feats

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occyp_type',
 'EMPLOYED_group']

In [14]:
list(train['EMPLOYED_group'].value_counts().index)

['0~5', '5~10', '10~15', '15~20', '20~25', '25~30', '30~35', '35~40', '40~']

In [15]:
for i in categorical_feats:
    print(f'{i} 컬럼 항목은 : {list(train[i].value_counts().index)}')

gender 컬럼 항목은 : ['F', 'M']
car 컬럼 항목은 : ['N', 'Y']
reality 컬럼 항목은 : ['Y', 'N']
income_type 컬럼 항목은 : ['Working', 'Commercial associate', 'Pensioner', 'State servant', 'Student']
edu_type 컬럼 항목은 : ['Secondary / secondary special', 'Higher education', 'Incomplete higher', 'Lower secondary', 'Academic degree']
family_type 컬럼 항목은 : ['Married', 'Single / not married', 'Civil marriage', 'Separated', 'Widow']
house_type 컬럼 항목은 : ['House / apartment', 'With parents', 'Municipal apartment', 'Rented apartment', 'Office apartment', 'Co-op apartment']
occyp_type 컬럼 항목은 : ['Laborers', 'No work experience', 'NAN', 'Core staff', 'Sales staff', 'Managers', 'Drivers', 'High skill tech staff', 'Accountants', 'Medicine staff', 'Cooking staff', 'Security staff', 'Cleaning staff', 'Private service staff', 'Low-skill Laborers', 'Waiters/barmen staff', 'Secretaries', 'Realty agents', 'HR staff', 'IT staff']
EMPLOYED_group 컬럼 항목은 : ['0~5', '5~10', '10~15', '15~20', '20~25', '25~30', '30~35', '35~40', '40~']


In [16]:
# 수치형 컬럼

# 'child_num' = 정수
# 'income_total' = 만원단위
# 'family_size' = 정수
# 'begin_month' = 달단위
# 'YEARS_BIRTH' = 나이
# 'income_avg' = income_total / famliy_size
# 'Age_group' = 
# def change_category(num):
#     return (math.trunc(num)//10)*10

# train['Age_group']=train['YEARS_BIRTH'].apply(change_category)
# test['Age_group']=test['YEARS_BIRTH'].apply(change_category)

# MODELING

In [17]:
seed = 42
n_fold_list =14
n_class = 3

target = 'credit'
X = train.drop(target, axis=1)
y = train[target]
X_test = test

## CatBoostClassifier

In [19]:
print('***',n_fold_list,'***')

skfold = StratifiedKFold(n_splits=n_fold_list, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = np.zeros((X_test.shape[0], n_class))
cat_cols = ['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type', 'EMPLOYED_group']

for fold in range(n_fold_list):
    print(f'\n----------------- Fold {fold} -----------------\n')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
    train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
    valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

    model_cat = CatBoostClassifier(n_estimators=3000, max_depth=8, random_seed=seed, learning_rate=0.04, 
                                bootstrap_type ='Bernoulli',auto_class_weights='SqrtBalanced')#,task_type='GPU')
    model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)

    cat_pred[valid_idx] = model_cat.predict_proba(X_valid)
    cat_pred_test += model_cat.predict_proba(X_test) / n_fold_list
    print(f'CV Log Loss Score: {log_loss(y_valid, cat_pred[valid_idx]):.6f}')
    
print(f'\t<*><*><*><*><*><*>Log Loss: {log_loss(y, cat_pred):.6f}<*><*><*><*><*><*>\n\n\n\n')

*** 14 ***

----------------- Fold 0 -----------------

0:	learn: 1.0892624	test: 1.0888253	best: 1.0888253 (0)	total: 105ms	remaining: 5m 16s
100:	learn: 0.9619043	test: 0.9629446	best: 0.9629446 (100)	total: 3.95s	remaining: 1m 53s
200:	learn: 0.9430534	test: 0.9568599	best: 0.9568599 (200)	total: 8.09s	remaining: 1m 52s
300:	learn: 0.9153855	test: 0.9468114	best: 0.9468114 (300)	total: 12.3s	remaining: 1m 49s
400:	learn: 0.8796654	test: 0.9328819	best: 0.9328819 (400)	total: 16.4s	remaining: 1m 46s
500:	learn: 0.8510514	test: 0.9245893	best: 0.9245893 (500)	total: 21s	remaining: 1m 44s
600:	learn: 0.8241047	test: 0.9172954	best: 0.9172954 (600)	total: 25.9s	remaining: 1m 43s
700:	learn: 0.7977793	test: 0.9117716	best: 0.9117716 (700)	total: 30s	remaining: 1m 38s
800:	learn: 0.7734567	test: 0.9079144	best: 0.9079144 (800)	total: 33.7s	remaining: 1m 32s
900:	learn: 0.7506303	test: 0.9031884	best: 0.9030773 (898)	total: 38.2s	remaining: 1m 28s
1000:	learn: 0.7294773	test: 0.8991279	bes

In [ ]:
def plot_feature_importance(importance,names,model_type):
    
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    plt.figure(figsize=(10,8))

    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])

    plt.title(model_type + ' Feature Importance')
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature Names')

In [ ]:
plot_feature_importance(model_cat.get_feature_importance(),X_test.columns,'CATBOOST')

In [ ]:
model_cat.save_model('cat_model')

model=CatBoostClassifier()

model.load_model('cat_model')

## XGboost

### categorical features OrdinalEncoder

In [ ]:
train.head()

In [ ]:
encoder = OrdinalEncoder(categorical_feats)
train[categorical_feats] = encoder.fit_transform(train[categorical_feats], train['credit'])
test[categorical_feats] = encoder.transform(test[categorical_feats])

In [ ]:
target = 'credit'
X = train.drop(target, axis=1)
y = train[target]
X_test = test

X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2)

In [ ]:
from sklearn.utils import class_weight
classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=y
)

### XGBoost Optuna

In [ ]:
def objective(trial):
    
    params = {
        "objective": "multi:softprob",
        "eval_metric":'mlogloss',
        "booster": 'gbtree',
        'tree_method':'gpu_hist', 'predictor':'gpu_predictor', 'gpu_id': 0, # GPU 사용시
        #"tree_method": 'exact', 'gpu_id': -1,  # CPU 사용시
        "verbosity": 0,
        'num_class':3,
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "learning_rate": trial.suggest_uniform('learning_rate', 0.0001, 0.99),
        'n_estimators': trial.suggest_int("n_estimators", 1000, 10000, step=100),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 1.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-2, 1),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-2, 1),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.05),     
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 15),
        "gamma": trial.suggest_float("gamma", 0.1, 1.0, log=True),
        # 'num_parallel_tree': trial.suggest_int("num_parallel_tree", 1, 500) 추가하면 느려짐.
    }


    model = xgb.XGBClassifier(**params,sample_weight=classes_weights, random_state = 42, enable_categorical=False)

    bst = model.fit(X_train, y_train,eval_set=[(X_valid,y_valid)], early_stopping_rounds=50, verbose=False)
    preds = bst.predict(X_valid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_valid, pred_labels)
    return accuracy

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

### XGBModeling

In [ ]:
xgb_params=study.best_trial.params

In [ ]:
print('***',n_fold_list,'***')

skfold = StratifiedKFold(n_splits=n_fold_list, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = np.zeros((X_test.shape[0], n_class))

for fold in range(n_fold_list):
    print(f'\n----------------- Fold {fold} -----------------\n')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
    evals=[(X_valid,y_valid)]
    model_cat = xgb.XGBClassifier(**xgb_params,sample_weight=classes_weights,random_state=seed)
    model_cat.fit(X_train,y_train,eval_set=evals, early_stopping_rounds=100, verbose=100)

    cat_pred[valid_idx] = model_cat.predict_proba(X_valid)
    cat_pred_test += model_cat.predict_proba(X_test) / n_fold_list
    print(f'CV Log Loss Score: {log_loss(y_valid, cat_pred[valid_idx]):.6f}')
    
print(f'\t<*><*><*><*><*><*>Log Loss: {log_loss(y, cat_pred):.6f}<*><*><*><*><*><*>\n\n\n\n')

In [ ]:
def plot_feature_importance(importance,names,model_type):
    
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    plt.figure(figsize=(10,8))

    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])

    plt.title(model_type + ' Feature Importance')
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature Names')

In [ ]:
plot_feature_importance(model_cat.feature_importances_,X_test.columns,'XGBOOST')

## Randomforest

### RandomForest Optuna

In [ ]:
def objective(trial):
    
    params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
            'max_depth': trial.suggest_int('max_depth', 1, 50),
            #'min_samples_split': trial.suggest_int('min_samples_split', 1, 150),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 60),
            'bootstrap' : trial.suggest_categorical('bootstrap',['True','False']),
            'max_features' :trial.suggest_categorical('max_features', ['auto', 'sqrt','log2']),
            'max_leaf_nodes' : trial.suggest_int('max_leaf_nodes', 1, 10000)
        }

    model = RandomForestClassifier(**params, random_state = 42,class_weight='balanced')

    bst = model.fit(X_train, y_train)
    preds = bst.predict(X_valid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_valid, pred_labels)
    return accuracy

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

### RFModeling

In [ ]:
randomforest_params=study.best_trial.params

In [ ]:
print('***',n_fold_list,'***')

skfold = StratifiedKFold(n_splits=n_fold_list, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = np.zeros((X_test.shape[0], n_class))
for fold in range(n_fold_list):
    print(f'\n----------------- Fold {fold} -----------------\n')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
    #evals=[(X_valid,y_valid)]
    model_cat = RandomForestClassifier(**randomforest_params,class_weight='balanced',random_state=seed)
    model_cat.fit(X_train,y_train)

    cat_pred[valid_idx] = model_cat.predict_proba(X_valid)
    cat_pred_test += model_cat.predict_proba(X_test) / n_fold_list
    print(f'CV Log Loss Score: {log_loss(y_valid, cat_pred[valid_idx]):.6f}')
    
print(f'\t<*><*><*><*><*><*>Log Loss: {log_loss(y, cat_pred):.6f}<*><*><*><*><*><*>\n\n\n\n')

In [ ]:
def plot_feature_importance(importance,names,model_type):
    
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    plt.figure(figsize=(10,8))

    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])

    plt.title(model_type + ' Feature Importance')
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature Names')

In [ ]:
plot_feature_importance(model_cat.feature_importances_,X_test.columns,'RandomForest')